https://minimin2.tistory.com/137

https://leo-bb.tistory.com/62?category=858291

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.std.html

# 결측치 많은 변수들

'user_test_correct_cnt_per_test': 372603,

'user_acc_per_test': 372603,

'user_total_acc': 365164,

 'user_total_correct_cnt': 365164

In [1]:
!pip install pycaret[full] ngboost shap

  Using cached shap-0.39.0.tar.gz (356 kB)
  ERROR: Command errored out with exit status 1:
   command: /opt/conda/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-e5dwqmo4/shap/setup.py'"'"'; __file__='"'"'/tmp/pip-install-e5dwqmo4/shap/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-0krri7ov
       cwd: /tmp/pip-install-e5dwqmo4/shap/
  Complete output (119 lines):
  The nvcc binary could not be located in your $PATH. Either  add it to your path, or set $CUDAHOME to enable CUDA
  Error building cuda module: TypeError('cannot unpack non-iterable NoneType object')
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.7
  creating build/lib.linux-x86_64-3.7/shap
  copying shap/__init__.py -> build/lib.linux-x86_64-3.7/shap
  copying shap/_

# LGBM with pycaret



In [2]:
import numpy as np
import pandas as pd
import os
import random
import pickle
from pycaret.classification import *
from pycaret.utils import check_metric
import time
from datetime import timedelta, timezone, datetime
from copy import deepcopy
from collections import defaultdict
from tqdm import tqdm_notebook, tqdm
import json
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

# 데이터 가져오기

In [3]:
upper_dir = '/opt/p4-dkt-freshtomato/input/data/train_dataset'
train_path = f'{upper_dir}/train_data.csv'
# train_path = f'{upper_dir}/new_train_data.csv'
test_path = f'{upper_dir}/test_data.csv'
submission_path = f'{upper_dir}/sample_submission.csv'

In [4]:
train = pd.read_csv(train_path) 
test = pd.read_csv(test_path)

answerCode2bool = {'userID':object,  'answerCode': 'int16', 'KnowledgeTag':object}
train = train.astype(answerCode2bool)
test = test.astype(answerCode2bool)
# train.Timestamp = pd.to_datetime(train.Timestamp)
# test.Timestamp = pd.to_datetime(test.Timestamp)

In [5]:
# all_data = True
# if all_data == True:
#     train = pd.concat([train,test],axis=0)
#     train = train.sort_values(['userID'])
#     train.index = range(train.shape[0])
print(train.shape)

(2266586, 6)


# Feature Engineering

## Train에서 미리 추출해야할 변수들

In [6]:
# # trian에서 각 문제별 통계량 추출
# # 아래의 피쳐들은 train에서 뽑은 값들을 test에 대입해주어야함.


total_num_prob_in_test = train[['assessmentItemID', 'testId']].drop_duplicates().groupby('testId').size()  
testid_maxlen_dict = total_num_prob_in_test.to_dict()

# LB Prediction해야하는 문항 ID들을 파악
set_assessmentItemID = set(test.loc[test.answerCode == -1, 'assessmentItemID'].values)
set_testID = set(test.loc[test.answerCode == -1, 'testId'].values)

## Input Data에 대해서 만들어줘야 하는 피쳐들

In [7]:
asset_dir = '/opt/p4-dkt-freshtomato/input/data/train_dataset/dict_for_feature'
# LB prediction에서 맞춰야하는 문항 ID 파악
def pd_join(data, column_name, value):
    value = pd.DataFrame(value, columns=[f"{column_name}"])
    data = data.join(value)
    return data
def save_pickle(path, values):
    os.makedirs(asset_dir, exist_ok=True)
    with open(f"{asset_dir}/{path}", 'wb') as f:
        pickle.dump(values, f)

def load_pickle(path):
    with open(f"{asset_dir}/{path}", 'rb') as f:
        load_values = pickle.load(f)
    return load_values


# Feature Engineering

In [8]:
def feature_engineering(df, is_train):
    if is_train:
        # 본 피쳐들은 train에서 얻어진 값이며 이를 저장해둡니다.
        testId_mean_sum_count = df.groupby(['testId'])['answerCode'].agg(['mean','sum','count']).to_dict()
        save_pickle("testId_mean_sum_count.pk",testId_mean_sum_count)

        assessmentItemID_mean_sum_count = df.groupby(['assessmentItemID'])['answerCode'].agg(['mean', 'sum','count']).to_dict()
        save_pickle("assessmentItemID_mean_sum_count.pk",assessmentItemID_mean_sum_count)

        KnowledgeTag_mean_sum_count = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum','count']).to_dict()
        save_pickle("KnowledgeTag_mean_sum_count.pk",KnowledgeTag_mean_sum_count)


    # 본 피쳐들은 train에서 얻어진 값을 그대로 test에도 사용합니다.
    # -> inference할 때는, 저장된 정답률을 가져와서 mapping하는 코드입니다.
    testId_mean_sum_count = load_pickle("testId_mean_sum_count.pk")
    df["testId_mean"] = df.testId.map(testId_mean_sum_count['mean'])
    df['testId_sum'] = df.testId.map(testId_mean_sum_count['sum'])

    assessmentItemID_mean_sum_count = load_pickle("assessmentItemID_mean_sum_count.pk")
    df["assessmentItemID_mean"] = df.assessmentItemID.map(assessmentItemID_mean_sum_count['mean'])
    df['assessmentItemID_sum'] = df.assessmentItemID.map(assessmentItemID_mean_sum_count['sum'])

    KnowledgeTag_mean_sum_count = load_pickle("KnowledgeTag_mean_sum_count.pk")
    df["tag_mean"] = df.KnowledgeTag.map(KnowledgeTag_mean_sum_count['mean'])
    df['tag_sum'] = df.KnowledgeTag.map(KnowledgeTag_mean_sum_count['sum'])

    ## 문제번호 (시험지 내에서) (범주형? 연속형?) - 3자리
    # 0 부터 시작하도록 
    df['problem_number'] = df.assessmentItemID.map(lambda x: int(x[-3:])-1)

    ## 시험지의(testID 별) 문제 최대개수가 몇인지 dict
    #  (이를 파악하여 중복 풀이하는 친구들을 걸러낼수있음)
    total_num_prob_in_test = df[['assessmentItemID', 'testId']].drop_duplicates().groupby('testId').size()
    testid_maxlen_dict = total_num_prob_in_test.to_dict()

    # 시험지별 문제번호 최대값
    max_prob_in_test = df.groupby('testId').problem_number.max()
    # 시험 문제번호의 최대값과 시험지 내의 문제개수가 일치하지 않는 시험지의 testId들을 추출
    inconsistent_index = max_prob_in_test[max_prob_in_test +1 != total_num_prob_in_test].index  # max_prob_in_test는 0부터 시작하고, num_prob_in_test는 1부터 시작
    # 문제 max번호와 문제개수가 일치하지 않는 시험지에 해당하는 데이터프레임으로부터 (시험지 Id와 문제 Id) 추출 후 문제 Id의 오름차순 순서대로 정렬
    inconsistent_df = df.loc[df.testId.isin(inconsistent_index),['assessmentItemID', 'testId']].drop_duplicates().sort_values('assessmentItemID')      

    # 순서대로 안 푼 시험지에서의 본래 문제의 순서가 dict에 저장된다    (inconsistent_Itemid_item_dict)
    inconsistent_Itemid_item_dict = {}
    inconsistent_df_group = inconsistent_df.groupby('testId')
    # 순서대로 안 푼 시험지들에 대하여 for문
    for key in inconsistent_df_group.groups:
        for i, (k,_) in enumerate(inconsistent_df_group.get_group(key).values):
            inconsistent_Itemid_item_dict[k] = i
    
    # origin_problem_order : 한 시험지 내에서 해당 문제 Id의 본래 순서 (중간에 비어있는 문제는 없는 문제로 생각하여 다시 순서를 매김)
    # ex) A080096003,A080096001,A080096005,A080096006,A080096007,A080096008,A080096002 
    # 문제번호 : 3, 1, 5, 6, 7, 8, 2 => 문제번호 중 4번이 빠져있음
    # 따라서 4번을 제외하고 문제번호를 본래순서에 대응시키면 => (1:0,2:1,3:2,5:3,6:4,7:5,8:6)
    # 변수생성 => origin_problem_order : (2,0,3,4,5,6,1)
    # 시험 문제번호의 최대값과 시험지 내의 문제개수가 일치하지 않는 시험지의 testId들을 추출        
    df['origin_problem_order'] =  df.assessmentItemID.map(lambda x: int(inconsistent_Itemid_item_dict[x]) if x in inconsistent_Itemid_item_dict else int(x[-3:]) - 1) # 0부터 시작하도록 -1 해줌.

    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    
    # 유저가 푼 시험지들에 대해, 유저의 누적 정답/풀이횟수/정답률 계산
    df_group = df.groupby(['userID','testId'])['answerCode']
    df['user_total_correct_cnt'] = df_group.transform(lambda x: x.cumsum().shift(1))    # 정답개수
    df['user_total_ans_cnt'] = df_group.cumcount()  # 풀이횟수
    df['user_total_acc'] = df['user_total_correct_cnt'] / df['user_total_ans_cnt']  # 정답률

    df['total_num_prob_in_test'] = df.testId.map(testid_maxlen_dict)    # 문제수
    # 특정 시험지를 몇번째 반복하여 풀고있는 것인지 계산
    df['nth_test'] = df['user_total_ans_cnt'] // df['total_num_prob_in_test'] # (같은)시험 완료 횟수
    # 유저가 풀고 있는 시험지에 대해, 현재 몇번째 문제인지
    df['user_test_ans_cnt'] = df['user_total_ans_cnt'] % df['total_num_prob_in_test']   # 현재 몇번째 문제인지

    # 각 시험지 당 유저의 정확도를 계산
    df['user_test_correct_cnt_per_test'] = df.groupby(['userID','testId','nth_test'])['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_acc_per_test'] = df['user_test_correct_cnt_per_test']/df['user_test_ans_cnt']

    df['int_KnowledgeTag'] = df.KnowledgeTag.astype('int64')
    df['test_total_id'] = df['assessmentItemID'].map(lambda x: int(x[1:7]))
    return df

In [9]:

# df = feature_engineering(train, is_train=True)
# print('train feature engineering Finished...')
df_test = feature_engineering(test, is_train=False)
print('test feature engineering Finished...')


test feature engineering Finished...


In [10]:
# Add Feature considering WindowSize
consider_window_features = ['answerCode']

def add_features_with_window(df,FEATS,window_size_list):
    empty_df = pd.DataFrame(index=range(df.shape[0]))
    for window_size in tqdm_notebook(window_size_list):
        add_df = df.groupby(['userID'])[FEATS].shift(window_size).copy().add_suffix(f'_shift{window_size}')
        empty_df = pd.concat([empty_df,add_df],axis=1)

    return pd.concat([df,empty_df],axis=1)

window_size_list=list(range(1,2))  # window size : 1 ~ 
# df = add_features_with_window(df,consider_window_features,window_size_list)
df_test = add_features_with_window(df_test,consider_window_features,window_size_list)
df_test

  0%|          | 0/1 [00:00<?, ?it/s]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,testId_mean,testId_sum,assessmentItemID_mean,assessmentItemID_sum,tag_mean,tag_sum,problem_number,origin_problem_order,user_total_correct_cnt,user_total_ans_cnt,user_total_acc,total_num_prob_in_test,nth_test,user_test_ans_cnt,user_test_correct_cnt_per_test,user_acc_per_test,int_KnowledgeTag,test_total_id,answerCode_shift1
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,0.560944,856,0.646789,141,0.641379,1023,0,0,NaN,0,NaN,7,0,0,NaN,NaN,2626,50023,NaN
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,0.560944,856,0.628440,137,0.641379,1023,1,1,1.0,1,1.000000,7,0,1,1.0,1.000000,2626,50023,1.0
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,0.560944,856,0.577982,126,0.670013,1535,2,2,2.0,2,1.000000,7,0,2,2.0,1.000000,2625,50023,1.0
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,0.560944,856,0.655963,143,0.670013,1535,3,3,2.0,3,0.666667,7,0,3,2.0,0.666667,2625,50023,0.0
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,0.560944,856,0.307339,67,0.568970,2314,5,5,2.0,4,0.500000,7,0,4,2.0,0.500000,2623,50023,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,0.644961,832,0.445736,115,0.645148,1529,0,0,NaN,0,NaN,5,0,0,NaN,NaN,8832,40130,1.0
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,0.644961,832,0.476744,123,0.645148,1529,1,1,0.0,1,0.000000,5,0,1,0.0,0.000000,8832,40130,0.0
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,0.644961,832,0.860465,222,0.767908,804,2,2,1.0,2,0.500000,5,0,2,1.0,0.500000,8244,40130,1.0
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244,0.644961,832,0.825581,213,0.767908,804,3,3,2.0,3,0.666667,5,0,3,2.0,0.666667,8244,40130,1.0


In [12]:
df_csv_name_after_FE = f'{upper_dir}/only_train_25개_원래변수들.csv'
# df.to_csv(df_csv_name_after_FE,index=False)
df = pd.read_csv(df_csv_name_after_FE)
print(f"df_csv_name_after_FE : {df_csv_name_after_FE}")

df_csv_name_after_FE : /opt/p4-dkt-freshtomato/input/data/train_dataset/only_train_25개_원래변수들.csv


In [13]:
print(df.shape)


(2266586, 25)


In [14]:
# LB 제출해야하는 testset
df_test_shift = df_test[df_test['userID'] != df_test['userID'].shift(-1)]

## 엔지니어링 이후의 EDA

In [15]:
check_df = df[df['userID'] != df['userID'].shift(-1)]
nan_check_per_column_dict = check_df.isnull().sum().to_dict()
print('User 별 마지막 row에 결측치가 존재하는 Column')
over_zero_nan_per_column = {k:v for k,v in nan_check_per_column_dict.items() if v!=0}
print(over_zero_nan_per_column)

check_df = df_test[df_test['userID'] != df_test['userID'].shift(-1)]
nan_check_per_column_dict = check_df.isnull().sum().to_dict()
print('User 별 마지막 row에 결측치가 존재하는 Column')
over_zero_nan_per_column = {k:v for k,v in nan_check_per_column_dict.items() if v!=0}
print(over_zero_nan_per_column)



User 별 마지막 row에 결측치가 존재하는 Column
{}
User 별 마지막 row에 결측치가 존재하는 Column
{}


In [16]:
check_df = df[df['userID'] != df['userID'].shift(1)]
nan_check_per_column_dict = check_df.isnull().sum().to_dict()
print('User 별 첫 row에 결측치가 존재하는 Column')
over_zero_nan_per_column = {k:v for k,v in nan_check_per_column_dict.items() if v!=0}
print(over_zero_nan_per_column)

check_df = df_test[df_test['userID'] != df_test['userID'].shift(1)]
nan_check_per_column_dict = check_df.isnull().sum().to_dict()
print('User 별 첫 row에 결측치가 존재하는 Column')
over_zero_nan_per_column = {k:v for k,v in nan_check_per_column_dict.items() if v!=0}
print(over_zero_nan_per_column)


User 별 첫 row에 결측치가 존재하는 Column
{'user_total_correct_cnt': 6698, 'user_total_acc': 6698, 'user_test_correct_cnt_per_test': 6698, 'user_acc_per_test': 6698, 'answerCode_shift1': 6698}
User 별 첫 row에 결측치가 존재하는 Column
{'user_total_correct_cnt': 744, 'user_total_acc': 744, 'user_test_correct_cnt_per_test': 744, 'user_acc_per_test': 744, 'answerCode_shift1': 744}


In [17]:
# 모든 데이터셋 row에 대하여 결측치가 1개 이상인 column, 결측치 개수
{k:v for k,v in df.isnull().sum().to_dict().items() if v!=0}

{'user_total_correct_cnt': 365164,
 'user_total_acc': 365164,
 'user_test_correct_cnt_per_test': 372603,
 'user_acc_per_test': 372603,
 'answerCode_shift1': 6698}

# Split Train valid Set

In [18]:
is_all_data = False
pre_split_beforehand = False # '', 'stratify_split' train과 holdout 나눌때 stratify 기준으로 먼저 나누고 시작하기
valid_only_LB = 'problem'   # '', 'problem', 'test', # prediction에 사용할 test row들의 (문제ID, 시험지ID)에 해당하는 row만 추출하여 Valid한다.
train_except_holdout = 'False'   # 'index', 'user' # holdout과 train을 겹치지 않게 할지, 그렇다면 그 기준은 무엇으로 할지
train_only_LB = 'problem'   # '', 'problem', 'test' # prediction에 사용할 test row들의 (문제ID, 시험지ID)에 해당하는 row만 추출하여 Train한다.
# train_with_finalrow = None
train_with_finalrow = 'False'  # None 'test','user' # 시험지 혹은 유저별 마지막 row만 가져와서 학습할 것인지
seed = 42

In [19]:
def train_holdout_split(
                        df,
                        is_all_data=is_all_data,  # input df가 train과 test 모두인지,
                        pre_split_beforehand=pre_split_beforehand,
                        valid_only_LB=valid_only_LB,
                        train_except_holdout=train_except_holdout,
                        train_with_finalrow = train_with_finalrow,
                        train_only_LB = train_only_LB,
                        ):

    # train과 test를 합쳐 사용한다면 answerCode==-1에 해당하는 row는 모두 지워준다.
    if is_all_data:
        copy_df = df[df.answerCode != -1].copy()
    else:
        copy_df = df.copy()
    print(copy_df.shape)
    if pre_split_beforehand == 'stratify_split':
        split_idx = list(dict.fromkeys(copy_df.userID))
        ## stratified split을 하기 위한 기준을 설정한다.
        stratify_key = copy_df.groupby(['userID'])['answerCode'].apply(lambda x: x.iloc[-1])
        tr_val_idx, holdout_idx = train_test_split(split_idx, test_size=0.2,random_state=seed,shuffle=True,stratify=stratify_key)
    else:
        tr_val_idx = (copy_df['userID'] == copy_df['userID'])
        holdout_idx = (copy_df['userID'] == copy_df['userID'])
    print(all(tr_val_idx))
    print(len(tr_val_idx))
    print(len(holdout_idx))

    # 유저별 마지막 row에 해당하는 데이터만 Holdout으로 사용
    # 추가적으로 LB Test set과 동일 조건의 row만 Valid에서 추출할지 결정
    # 빈 값이 아니라면, 추가조건으로 리더보드와 동일 문제ID에 해당하는 row만 추출하여 holdout로 사용
    if valid_only_LB=='problem':
        holdout_filtered_idxes = (copy_df['userID'] != copy_df['userID'].shift(-1)) & (copy_df.assessmentItemID.isin(set_assessmentItemID)) & holdout_idx
    elif valid_only_LB=='test':
        holdout_filtered_idxes = (copy_df['userID'] != copy_df['userID'].shift(-1)) & (copy_df.testId.isin(set_testID)) & holdout_idx
    else:
        holdout_filtered_idxes = (copy_df['userID'] != copy_df['userID'].shift(-1)) & holdout_idx

    holdout = copy_df[holdout_filtered_idxes].copy()

    print(f"set(holdout.userID) : {len(set(holdout.userID))}")
    print(f"set(holdout.index) : {len(set(holdout.index))}")
    if train_except_holdout=='user':
        tr_val_filtered_idxes = (copy_df['userID'].isin(set(holdout.userID)) == False) & tr_val_idx
    elif train_except_holdout=='index':
        tr_val_filtered_idxes = (copy_df.index.isin(set(holdout.index)) == False) & tr_val_idx
    else:
        tr_val_filtered_idxes = tr_val_idx
    
    tr_val = copy_df[tr_val_filtered_idxes].copy()

    print(f"(tr_val.shape) : {tr_val.shape}")

    # 학습시 무엇을 기준으로 마지막 row만 가져올지
    if train_with_finalrow == 'test':
        tr_val = tr_val[tr_val['testId'] != tr_val['testId'].shift(-1)].copy()
    elif train_with_finalrow == 'user':
        tr_val = tr_val[tr_val['userID'] != tr_val['userID'].shift(-1)].copy()
    else:
        pass

    # LB prediction의 row에서 맞혀야하는 문제들만 학습할 것인지 (444개)
    if train_only_LB=='problem':
        tr_val = tr_val[tr_val.assessmentItemID.isin(set_assessmentItemID)].copy()
    # LB prediction의 row에서 맞혀야하는 시험지들만 학습할 것인지 (411개)
    elif train_only_LB=='test':
        tr_val = tr_val[tr_val.testId.isin(set_testID)].copy()
    else:
        pass

    return tr_val, holdout

In [20]:
fold = 10
method = 'soft' # 'soft' , 'hard' : 모델들 blend할 때, 앙상블방법

tr_val, holdout = train_holdout_split(
                                    df, 
                                    is_all_data=is_all_data,
                                    pre_split_beforehand=pre_split_beforehand,
                                    valid_only_LB=valid_only_LB,
                                    train_except_holdout=train_except_holdout,
                                    train_with_finalrow = train_with_finalrow,
                                    train_only_LB = train_only_LB,
                                    )

print('====최종적인 len(tr_val), len(holdout)====')
print(len(tr_val), len(holdout))
print(tr_val.answerCode.mean(), holdout.answerCode.mean())


(2266586, 25)
True
2266586
2266586
set(holdout.userID) : 4775
set(holdout.index) : 4775
(tr_val.shape) : (2266586, 25)
====최종적인 len(tr_val), len(holdout)====
107202 4775
0.5593832204623048 0.47706806282722514


# AUTOML (NGBoosting도 쓸 수 있으니 참고)

## 모델, grid score df, plot, config, output csv 파일들이 저장될 상위 경로

In [21]:
# 모델들, grid score df, plot 등이 모두 저장되는 상위경로
now_time = datetime.now(timezone(timedelta(hours=9))).strftime('%m%d_%H%M')
upper_dir_of_pycaret = f'/opt/p4-dkt-freshtomato/output_pycaret/{now_time}'
print(f"upper_dir_of_pycaret : {upper_dir_of_pycaret}")

upper_dir_of_pycaret : /opt/p4-dkt-freshtomato/output_pycaret/0614_0553


##  각 단계마다 (create - tune - ensemble - finalize) 모델, 정보 저장해주는 코드

In [22]:
def plot_save_pycaret_model(name, model,mode='tune'):
    # Feature Importance Plot 저장하기
    plot_dir = f'{upper_dir_of_pycaret}/plot_result'
    os.makedirs(plot_dir, exist_ok=True)
    os.chdir(plot_dir)
    # top10 feature
    feature_importance_plot = plot_model(model, plot = 'feature', save=True)
    os.rename(f'{plot_dir}/Feature Importance.png',f'{plot_dir}/{mode}_{name}_10_feature_importance.png')
    # all feature
    feature_importance_plot = plot_model(model, plot = 'feature_all', save=True)
    os.rename(f'{plot_dir}/Feature Importance (All).png',f'{plot_dir}/{mode}_{name}_all_feature_importance.png')

# def create_and_save_pycaret_model(name):
#     created_model = create_model(name, cross_validation = True)
#     # Display되는 score grid dataframe도 가져올 수가 있다. 
#     df_result_model = pull()
#     df_result_model.to_csv(f'{upper_dir_of_pycaret}/{name}_create_grid_df.csv')
#     # 실험결과를 저장하는 것도 가능하다.
#     save_experiment(f'{upper_dir_of_pycaret}/{name}_create_exp_result')
#     # 모델을 저장하는 것도 가능하다.
#     save_model(created_model, f'{upper_dir_of_pycaret}/{name}_model_saved')
#     return created_model

def tune_and_save_pycaret_model(name, model, optimize = 'AUC', fold = fold, n_iter = 10):
    print('Now Tuning the models....')
    tuned_model = tune_model(model, optimize = optimize, fold = fold,)   # n_iter = n_iter
    # Display되는 score grid dataframe도 가져올 수가 있다. 
    df_result_model = pull()
    os.makedirs(f"{upper_dir_of_pycaret}/grid_df",exist_ok=True)
    df_result_model.to_csv(f'{upper_dir_of_pycaret}/grid_df/{name}_tuned_grid_df.csv')
    # 모델의 각종 plot을 저장하는 것도 가능하다.
    plot_save_pycaret_model(name, tuned_model, mode='tune')
    # 실험결과를 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/expr",exist_ok=True)
    try:
        save_experiment(f'{upper_dir_of_pycaret}/expr/{name}_tuned_exp_result')
    except:
        print('tuned model의 실험결과는 저장할 수 없습니다.')
    # 모델을 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/models",exist_ok=True)
    save_model(tuned_model, f'{upper_dir_of_pycaret}/models/{name}_tuned_model_saved')
    return tuned_model

def ensemble_and_save_pycaret_model(
                                    model_name_list,
                                    estimator_list,
                                    fold = fold,
                                    optimize = 'AUC',
                                    method = method,
                                    meta_model = None,
                                    mode = None,
                                    ):
    print('Now Ensemble the models....')
    if mode=='blend_models':
        ensembled_model = blend_models(estimator_list = estimator_list, fold = fold, optimize = optimize, method = method)
    elif mode=='stack_models':
        ensembled_model = stack_models(estimator_list = estimator_list, fold = fold, optimize = optimize, meta_model = meta_model)
    # Display되는 score grid dataframe도 가져올 수가 있다. 
    df_result_model = pull()
    name = '_'.join(model_name_list)
    os.makedirs(f"{upper_dir_of_pycaret}/grid_df",exist_ok=True)
    df_result_model.to_csv(f'{upper_dir_of_pycaret}/grid_df/{name}_ensembled_grid_df.csv')
    # 모델의 각종 plot을 저장하는 것도 가능하다.
    try:
        plot_save_pycaret_model(name, ensembled_model, mode='tune')
    except:
        print(f'ensemble model은 변수중요도 plot을 저장할 수 없습니다')
    # 실험결과를 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/expr",exist_ok=True)
    try:
        save_experiment(f'{upper_dir_of_pycaret}/expr/{name}_ensembled_exp_result')
    except:
        print(f'ensemble model은 실험결과를 저장할 수 없습니다')
    # 모델을 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/models",exist_ok=True)
    save_model(ensembled_model, f'{upper_dir_of_pycaret}/models/{name}_ensembled_model_saved')
    return ensembled_model

def finalize_and_save_pycaret_model(model_name_list,model,):
    print('Now Finalizing the model....')
    finalized_model = finalize_model(model)
    # Display되는 score grid dataframe도 가져올 수가 있다. 
    df_result_model = pull()
    name = '_'.join(model_name_list)
    os.makedirs(f"{upper_dir_of_pycaret}/grid_df",exist_ok=True)
    df_result_model.to_csv(f'{upper_dir_of_pycaret}/grid_df/{name}_finalized_grid_df.csv')
    # 모델의 각종 plot을 저장하는 것도 가능하다.
    try:
        plot_save_pycaret_model(name, finalized_model, mode='tune')
    except:
        print(f'final model은 변수중요도 plot을 저장할 수 없습니다')
    # 실험결과를 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/expr",exist_ok=True)
    try:
        save_experiment(f'{upper_dir_of_pycaret}/expr/{name}_finalized_exp_result')
    except:
        print(f'final model은 실험결과를 저장할 수 없습니다')
    # 모델을 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/models",exist_ok=True)
    save_model(finalized_model, f'{upper_dir_of_pycaret}/models/{name}_finalized_model_saved')
    return finalized_model


In [23]:
stratified=True
class LastSolveKFold:
    def __init__(self,
                n_splits=fold,
                shuffle=True,
                random_state=42
                ):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def split(self, X, y=None, groups=None):
        # print(X.head())
        # print(X['userID'])
        user_id = np.array(tr_val['userID'])
        X, y, groups = [np.array(iterable) for iterable in (X, y, groups)]
        n_samples = len(X)
        #########################################
        last_user_index = np.where(np.roll(user_id, shift=-1) != user_id)[0]
        stratify_key = tr_val.iloc[last_user_index].answerCode
        # stratify_key = tr_val.groupby(['userID'])['answerCode'].mean().apply(lambda x: 1 if x>0.61 else 0)
        total_val_indices = []
        total_indices = np.arange(n_samples)
        print(f"last_user_index : {len(last_user_index)}")
        print(f"total_indices : {len(total_indices)}")
        print(f"len(total_indices)-len(last_user_index)//10 : {len(total_indices)-len(last_user_index)//10}")
        if stratified==True:
            kfold_split = StratifiedKFold(n_splits=self.n_splits, shuffle=self.shuffle, random_state = self.random_state).split(last_user_index,stratify_key)
        else:
            kfold_split = KFold(n_splits=self.n_splits, shuffle=self.shuffle, random_state = self.random_state).split(last_user_index)

        for _, test_idx in tqdm(kfold_split):
            test_idx = last_user_index[test_idx]
            total_val_indices.extend(test_idx)
            # train_idx = [tr_id for tr_id in total_indices if not tr_id in test_idx]
            train_idx = np.setdiff1d(total_indices, test_idx)
            print(f'train_idx : {len(train_idx)}')
            # print(f'train_idx : {type(train_idx)}')
            print(f'val_idx : {len(test_idx)}')
            # print(f'val_idx : {type(test_idx)}')
            # print(f'val_idx : {test_idx}')
            print(f'shape of X : {X.shape}')
            yield np.array(train_idx), np.array(test_idx)
        print(f"valid Set이 된 모든 row들의 개수 : {len(total_val_indices)}")
        print(f"valid Set이 된 모든 row들 unique 개수 : {len(set(total_val_indices))}")

    def get_n_splits(self, X, y=None, groups=None):
        return self.n_splits

last_fold = LastSolveKFold(n_splits=10, shuffle=True, random_state=42)


# Pycaret tabular.py setup 함수 내부 수정
train_size에 상관없이 모든 데이터를 사용하도록

In [24]:
df.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'testId_mean', 'testId_sum', 'assessmentItemID_mean',
       'assessmentItemID_sum', 'tag_mean', 'tag_sum', 'problem_number',
       'origin_problem_order', 'user_total_correct_cnt', 'user_total_ans_cnt',
       'user_total_acc', 'total_num_prob_in_test', 'nth_test',
       'user_test_ans_cnt', 'user_test_correct_cnt_per_test',
       'user_acc_per_test', 'int_KnowledgeTag', 'test_total_id',
       'answerCode_shift1'],
      dtype='object')

In [27]:
experiment_name = 'LGBM-mjh-21_feature_Random-10fold'
FEATS = ['answerCode', 'Timestamp',
       'KnowledgeTag', 'problem_number', 'origin_problem_order',
       'user_total_correct_cnt', 'user_total_ans_cnt', 'user_total_acc',
       'total_num_prob_in_test', 'nth_test', 'user_test_ans_cnt',
       'user_test_correct_cnt_per_test', 'user_acc_per_test', 'testId_mean',
       'testId_sum', 'assessmentItemID_mean', 'assessmentItemID_sum', 'tag_mean', 'tag_sum',
       'int_KnowledgeTag', 'test_total_id']

cat_features=[]
continuous_features=[]

print(f'FEATS : {len(FEATS)}')
print(f"target : 'answerCode'")
print(f'cat_features : {len(cat_features)}')
print(f'continuous_features : {len(continuous_features)}')
print(f"etc_features : {[f for f in FEATS if f not in cat_features+continuous_features+['answerCode']]}")

## Classification Models :  https://github.com/pycaret/pycaret/blob/master/pycaret/containers/models/classification.py
## Regression Modelss : https://github.com/pycaret/pycaret/blob/master/pycaret/containers/models/regression.py
## scikit-learn 기반 모델이면 다음과 같이 다 써볼수가 있음.
settings = setup(
                data=tr_val[FEATS], 
                target='answerCode', 
                train_size=0.9, 
                categorical_features=cat_features, 
                numeric_features=continuous_features,
                fold_strategy = last_fold,
                data_split_shuffle = False, # True가 default
                # fix_imbalance=True,   # True로 하면 SMOTE를 기본으로 사용
                # experiment_name=experiment_name,
                # log_experiment=False,   # compare_model 할 때에는 하이퍼파라미터가 어떻게 튜닝되었는지 볼 수 있어서 유용하다고 한다.
                )

# from ngboost import NGBClassifier
# ngc = NGBClassifier()
# ngboost = create_model(ngc)

model_name_list=['lightgbm']   # lightgbm, gbc, lda, catboost, et, ada, rf, lr, nb, knn, qda, dt, xgboost, ngboost
ensemble_method = 'blend_models'  # 'blend_models', 'stack_models' # (Classifier : 'stack_models'), (Regression : 'create_stacknet')
meta_model_for_stack = 'xgboost' # If using stack_models


,Description,Value
0,session_id,7734
1,Target,answerCode
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(107202, 21)"
5,Missing Values,True
6,Numeric Features,13
7,Categorical Features,6
8,Ordinal Features,False
9,High Cardinality Features,False


In [28]:
print(f'is_all_data : {is_all_data}')
print(f'valid_only_LB : {valid_only_LB}')
print(f'train_except_holdout : {train_except_holdout}')
print(f'train_only_LB : {train_only_LB}')
print(f'train_with_finalrow : {train_with_finalrow}')
print(f'pre_split_beforehand : {pre_split_beforehand}')
# print(f'df.columns : {df.columns}') 
df.columns  # FEATS 참고용

is_all_data : False
valid_only_LB : test
train_except_holdout : False
train_only_LB : problem
train_with_finalrow : False
pre_split_beforehand : False


Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'testId_mean', 'testId_sum', 'assessmentItemID_mean',
       'assessmentItemID_sum', 'tag_mean', 'tag_sum', 'problem_number',
       'origin_problem_order', 'user_total_correct_cnt', 'user_total_ans_cnt',
       'user_total_acc', 'total_num_prob_in_test', 'nth_test',
       'user_test_ans_cnt', 'user_test_correct_cnt_per_test',
       'user_acc_per_test', 'int_KnowledgeTag', 'test_total_id',
       'answerCode_shift1'],
      dtype='object')

In [29]:
def ensemble_automl(
                    tr_val,
                    holdout,
                    FEATS,
                    cat_features=[],
                    continuous_features=[],
                    seed=seed,
                    model_name_list = model_name_list,
                    ensemble_method = ensemble_method,
                    meta_model_for_stack = meta_model_for_stack,
                    settings = settings,
                    ):

    # tr_val, holdout = datasets
    random.seed(seed)
    # fold_strategy = KFold(n_splits=10, shuffle=True, random_state=42)
    settings = settings

    # train_size만큼을 가지고 선언된 model_name_list 모델들을 학습을 함
    # create_model(name, sort='AUC', cross_validation = True)
    models_before_tune = [
                            create_model(
                                name, 
                                fold=fold,
                                cross_validation = True,
                                # cross_validation = False,
                                ) 
                            for name in model_name_list]
    # 앞서만든 모델들을 train_size만큼 가지고 튜닝함 (n_iter만큼 AutoML)
    # models_after_tune = [tune_model(model, optimize = 'AUC', fold = 10, n_iter = 10) for model in models_before_tune]
    models_after_tune = [tune_and_save_pycaret_model(name, model, optimize = 'AUC', fold = fold, n_iter = 10) for name, model in zip(model_name_list, models_before_tune)]


    # 튜닝된 모델들을 train_size만큼 가지고 앙상블
    if len(models_after_tune)<=1:
        ensembled = models_after_tune[0]
    else:
        ensembled = ensemble_and_save_pycaret_model(model_name_list,estimator_list=models_after_tune,fold = fold,optimize = 'AUC',method = method,meta_model = meta_model_for_stack,mode = ensemble_method)
    # elif ensemble_method=='blend_models':
    #     ensembled = blend_models(estimator_list = models_after_tune, fold = 10, method = 'soft', optimize = 'AUC')
    # elif ensemble_method=='stack_models':
    #     ensembled = stack_models(estimator_list = models_after_tune, meta_model = meta_model_for_stack, fold = 10, optimize = 'AUC')

    # 마지막 학습(Finalize)
    # 앞서 앙상블된 모델을 => setup으로 나눠져 쓰지않았던 valid까지 포함된 100퍼센트를 사용하여 fitting함
    final_model = finalize_and_save_pycaret_model(model_name_list,ensembled)

    metric_result = []
    prediction = predict_model(final_model, data=holdout[FEATS], raw_score = True)
    df_holdout_score = pull()

    os.makedirs(f"{upper_dir_of_pycaret}/holdout_score",exist_ok=True)
    df_holdout_score.to_csv(f'{upper_dir_of_pycaret}/holdout_score/finalize_holdout_score.csv')

    metric_result.append(f"HoldOut 데이터 ACC & AUC: {check_metric(prediction['answerCode'], prediction['Label'], metric = 'Accuracy')} ,{check_metric(prediction['answerCode'], prediction['Score_1'], metric = 'AUC')}")
    return final_model, metric_result

In [30]:
final_model, metric_result = ensemble_automl(
                                            tr_val,
                                            holdout,
                                            FEATS,
                                            cat_features,
                                            continuous_features,
                                            seed=seed,
                                            model_name_list=model_name_list,
                                            ensemble_method = ensemble_method,
                                            meta_model_for_stack = meta_model_for_stack,
                                            )

print(final_model)

print(f"ACC & AUC : ",'\n'.join(metric_result))
# 0.841 # 0.8124    # 0.8169

final model은 실험결과를 저장할 수 없습니다
Transformation Pipeline and Model Succesfully Saved
LGBMClassifier(bagging_fraction=0.5, bagging_freq=4, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=1.0,
               importance_type='split', learning_rate=0.05, max_depth=-1,
               min_child_samples=61, min_child_weight=0.001, min_split_gain=0.8,
               n_estimators=70, n_jobs=-1, num_leaves=70, objective=None,
               random_state=7734, reg_alpha=0.5, reg_lambda=4, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
ACC & AUC :  HoldOut 데이터 ACC & AUC: 0.737 ,0.8145


In [31]:
# DL 팀에서 사용하고 있는 K-Fold CV 전략 사용, Score 비교
va_acc_list = []
va_auc_list = []
df_all_idxes = list(dict.fromkeys(df.userID))
# stratify_key = df.groupby(['userID'])['answerCode'].apply(lambda x: x.iloc[-1])

kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold_counter = 0
for train_idx, val_idx in tqdm(kf.split(df_all_idxes)):
    train_idx = np.array(df_all_idxes)[train_idx]
    val_idx = np.array(df_all_idxes)[val_idx]

    # print(f"=========({fold_counter})=========")
    tr = df.loc[df.userID.isin(train_idx),:].copy()
    va = df.loc[df.userID.isin(val_idx),:].copy()
    # print(f'df.shape : {df.shape}')
    # print(f'tr.shape : {tr.shape}')
    # print(f'va.shape : {va.shape}')

    # print(len(set(train_idx)),len(set(val_idx)))
    # print(f'df.userID : {len(set(df.userID))}')
    # print(f'tr.userID : {len(set(tr.userID))}')
    # print(f'va.userID : {len(set(va.userID))}')

    # 예측할 데이터들은 user별 마지막 행
    rows_for_predict = va.groupby(['userID'], as_index = False).last().copy()
    # print(f'rows_for_predict : {rows_for_predict.shape}')
    va_pred = predict_model(final_model, data=rows_for_predict[FEATS], raw_score = True)
    va_acc = check_metric(va_pred['answerCode'], va_pred['Label'], metric = 'Accuracy')
    va_acc_list.append(va_acc)

    va_auc = check_metric(va_pred['answerCode'], va_pred['Score_1'], metric = 'AUC')
    va_auc_list.append(va_auc)
    fold_counter+=1

baseline_10fold_acc = np.mean(va_acc_list)
baseline_10fold_auc = np.mean(va_auc_list)
print(f'10-Fold ACC : {baseline_10fold_acc}')
print(f'10-Fold AUC : {baseline_10fold_auc}')

# 0.83359   # 0.8379    # 0.8145

10it [00:20,  2.09s/it]

10-Fold ACC : 0.7330600000000002
10-Fold AUC : 0.8145300000000001




# Inference

In [32]:
# 이번 Experiment Arg 관리
use_last_fold = True
config_key = [
            'experiment_name','is_all_data','valid_only_LB','train_except_holdout','train_only_LB','train_with_finalrow',
            'seed','fold','method','model_name_list','ensemble_method','meta_model_for_stack',
            'cat_features','continuous_features','FEATS','pre_split_beforehand','use_last_fold'
            ]
config_value = [
                experiment_name, is_all_data, valid_only_LB, train_except_holdout, train_only_LB, train_with_finalrow, 
                seed, fold,method, model_name_list, ensemble_method, meta_model_for_stack, 
                cat_features, continuous_features, FEATS, pre_split_beforehand,use_last_fold]


In [33]:
# MAKE PREDICTION
prediction = predict_model(final_model, data=df_test_shift[FEATS], raw_score=True)
total_preds = prediction.Score_1.values

# SAVE OUTPUT
prediction_name = f"{now_time}_피쳐{len(FEATS)}개사용_{experiment_name}" # 

output_dir = f'{upper_dir_of_pycaret}/submission'
os.makedirs(output_dir, exist_ok=True)    
write_path = os.path.join(output_dir, f"{prediction_name}.csv")
print("Saving Final Output Csv...")
with open(write_path, 'w', encoding='utf8') as w:
    print(f"writing prediction : {write_path}")
    w.write("id,prediction\n")
    for id, p in enumerate(total_preds):
        w.write('{},{}\n'.format(id,p))

# Save Config
write_path = os.path.join(output_dir, f"{prediction_name}_config.json")
config_dict = {k:v for k,v in zip(config_key, config_value)}
print("Saving Final Config Dict...")
## json파일 저장 ##
with open(write_path, "w") as fp:
    print(f"writing config : {write_path}")
    json.dump(config_dict, fp, indent=4)

Saving Final Output Csv...
writing prediction : /opt/p4-dkt-freshtomato/output_pycaret/0614_0553/submission/0614_0553_피쳐21개사용_LGBM-mjh-21_feature_Random-10fold.csv
Saving Final Config Dict...
writing config : /opt/p4-dkt-freshtomato/output_pycaret/0614_0553/submission/0614_0553_피쳐21개사용_LGBM-mjh-21_feature_Random-10fold_config.json
